In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('https://github.com/ridart44/Coca-cola-and-Pepsi-Stock-Price/blob/master/Pepsi.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 79, saw 6


In [ ]:

df.head

In [26]:
print(df.isnull)

<bound method DataFrame.isnull of             Date     Open     High      Low    Close     Adj Close      Volume
0       1/4/2000   6000.0   6110.0   5660.0   6110.0      0.090599  74195000.0
1       1/5/2000   5800.0   6060.0   5520.0   5580.0      0.082740  74680000.0
2       1/6/2000   5750.0   5780.0   5580.0   5620.0      0.083333  54390000.0
3       1/7/2000   5560.0   5670.0   5360.0   5540.0      0.082147  40305000.0
4      1/10/2000   5600.0   5770.0   5580.0   5770.0      0.085558  46880000.0
...          ...      ...      ...      ...      ...           ...         ...
5011  11/29/2019  51200.0  51400.0  50200.0  50300.0  50300.000000  11179334.0
5012   12/2/2019  50900.0  51300.0  50400.0  50400.0  50400.000000  18877252.0
5013   12/3/2019  49800.0  50300.0  49500.0  49900.0  49900.000000  12059812.0
5014   12/4/2019  49600.0  49850.0  49000.0  49450.0  49450.000000  13246549.0
5015   12/5/2019  50200.0  50400.0  49500.0  49500.0  49500.000000  10018229.0

[5016 rows x 7 co